In [15]:
# Import relevant libraries
import pandas as pd

In [16]:
# Import the master database
df = pd.read_csv('../data/new/master_final.csv')

df.describe()

,answer_id,user_id,school_id,user_level,question_id,difficulty,topic_id,subject_id,axis_id,guide_id,...,student_subject_attempts,student_subject_correct,student_axis_attempts,student_axis_correct,question_attempts_count,question_success_count,question_success_ratio,avg_question_time,user_level_percentile,student_age
count,1.960830e+05,196083.000000,167553.000000,196083.000000,196083.000000,196083.000000,196083.000000,196083.000000,196083.000000,196083.000000,...,196083.000000,196083.000000,196083.000000,196083.000000,196083.000000,196083.000000,196083.000000,196083.000000,196083.000000,166972.000000
mean,1.461729e+06,37926.608044,97.705717,-0.422881,22661.937501,-1.290956,525.486717,145.025836,23.741110,394082.718288,...,20.169571,15.066956,61.117415,46.437794,34.567994,24.812166,0.698035,57.094943,49.948445,15.576923
std,8.077267e+04,15973.799295,64.290289,1.594154,4073.089612,1.161867,68.331395,32.878704,0.945592,14126.804790,...,25.020991,19.622534,73.840500,60.747589,36.697974,28.596410,0.238587,42.779977,29.042479,1.872362
min,1.321329e+06,2315.000000,1.000000,-3.000000,15654.000000,-3.000000,409.000000,113.000000,23.000000,371032.000000,...,1.000000,0.000000,1.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,10.000000
25%,1.392200e+06,22057.000000,57.000000,-1.662527,19264.000000,-2.067897,462.000000,116.000000,23.000000,381112.000000,...,5.000000,3.000000,15.000000,10.000000,9.000000,5.000000,0.555556,31.138650,25.000000,14.000000
50%,1.462145e+06,50099.000000,91.000000,-0.518800,23449.000000,-1.410794,540.000000,128.000000,23.000000,393880.000000,...,12.000000,8.000000,38.000000,27.000000,23.000000,15.000000,0.741379,49.055804,50.000000,16.000000
75%,1.530326e+06,51893.000000,105.000000,0.648684,26200.000000,-0.932703,572.000000,181.000000,24.000000,406650.000000,...,26.000000,19.000000,80.000000,60.000000,47.000000,33.000000,0.883528,72.066864,75.000000,17.000000
max,1.603613e+06,53189.000000,238.000000,3.000000,29256.000000,2.994811,667.000000,199.000000,26.000000,419681.000000,...,313.000000,221.000000,817.000000,768.000000,274.000000,225.000000,1.000000,596.291865,97.000000,18.000000


In [17]:
# Print initial number of rows
print(f"Initial number of rows: {len(df)}")

# Group by user_id and topic_id and count rows
grouped_counts = df.groupby(['user_id', 'topic_id']).size()

# Convert to dataframe and filter for counts >= 10
filtered_groups = grouped_counts[grouped_counts >= 10].reset_index()

# Merge back with original dataframe to keep only rows meeting criteria
df = df.merge(filtered_groups[['user_id', 'topic_id']], on=['user_id', 'topic_id'])

# Print final number of rows
print(f"Final number of rows: {len(df)}")



Initial number of rows: 196083
Final number of rows: 93588


In [20]:
# Create sample meeting criteria
sample_df = (df
    # First get unique user-topic combinations
    .drop_duplicates(subset=['user_id', 'topic_id'])
    
    # Stratify by axis_id and is_correct
    .groupby(['axis_id', 'is_correct'])
    .apply(lambda x: x.sample(n=min(25, len(x)), random_state=42))
    .reset_index(drop=True)
    
    # Take first 200 rows after stratification
    .head(200)
)

# Verify distributions
print("\nAxis ID distribution:")
print(sample_df['axis_id'].value_counts(normalize=True))

print("\nCorrect answer distribution:")
print(sample_df['is_correct'].value_counts(normalize=True))

print("\nUser level distribution:")
print(sample_df['user_level'].describe())

# Verify uniqueness
print("\nUnique values:")
print(f"Users: {sample_df['user_id'].nunique()}")
print(f"Topics: {sample_df['topic_id'].nunique()}")



# Save sample to CSV
sample_df.to_csv('../data/new/sample_50.csv', index=False)



Axis ID distribution:
axis_id
23    0.25
24    0.25
25    0.25
26    0.25
Name: proportion, dtype: float64

Correct answer distribution:
is_correct
False    0.5
True     0.5
Name: proportion, dtype: float64

User level distribution:
count    200.000000
mean      -0.293022
std        1.516375
min       -3.000000
25%       -1.472716
50%       -0.199829
75%        0.680723
max        3.000000
Name: user_level, dtype: float64

Unique values:
Users: 159
Topics: 86


/var/folders/_q/rqr7_q191gx4ps32w9wtkmgw0000gn/T/ipykernel_33465/1392716697.py:8: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda x: x.sample(n=min(25, len(x)), random_state=42))


In [21]:
# Select and rename columns for export
columns_mapping = {
    'answer_id': 'answerId',
    'is_correct': 'isCorrect', 
    'question_title': 'questionTitle',
    'correct_option': 'correctOptionLetter',
    'student_answer': 'studentAnswerLetter',
    'student_age': 'studentAge',
    'user_level': 'userLevel',
    'user_level_percentile': 'userLevelPercentile',
    'topic_name': 'topicName',
    'subject_name': 'subjectName',
    'axis_name': 'axisName',
    'option_a': 'optionA',
    'option_b': 'optionB',
    'option_c': 'optionC',
    'option_d': 'optionD',
    'option_e': 'optionE'
}

# Select only mapped columns and rename them
export_df = sample_df[columns_mapping.keys()].rename(columns=columns_mapping)

# Export to CSV
export_df.to_csv('../data/new/sample_200_renamed.csv', index=False)

